In [3]:
from ipyaladin import Aladin

from ipywidgets import Layout, Box, widgets

CWB = Aladin(layout=Layout(width='50%'), survey='P/Mellinger/color',target='200 80', fov=140)
LIB = Aladin(layout=Layout(width='50%'), survey='P/Mellinger/color')
BAYESTAR = Aladin(layout=Layout(width='50%'), survey='P/Mellinger/color')
LALInference = Aladin(layout=Layout(width='50%'), survey='P/Mellinger/color')

# synchronize target between 3 widgets
widgets.jslink((CWB, 'target'), (LIB, 'target'))
widgets.jslink((LIB, 'target'), (BAYESTAR, 'target'))
widgets.jslink((BAYESTAR, 'target'), (LALInference, 'target'))

# synchronize FoV (zoom level) between 3 widgets
widgets.jslink((CWB, 'fov'), (LIB, 'fov'))
widgets.jslink((LIB, 'fov'), (BAYESTAR, 'fov'))
widgets.jslink((BAYESTAR, 'fov'), (LALInference, 'fov'))

items = [CWB, LIB, BAYESTAR, LALInference]

box_layout = Layout(display='flex',
                    flex_flow='wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box

A Jupyter Widget

In [4]:
import healpy as hp
import numpy as np
from mocpy import MOC

from math import log

class MOC_confidence_region(object):
    """
    Multi-Order coverage map (MOC) of sky areas enclosed within a contour plot at a given confidence level.
    """

    def read_skymap(self, infile):
        """Reading healpix skymap.
        
        Input parameters
        ----------------
        infile : string
              LVC probability sky localization in healpix format
              
        Return
        -------
        hpx : list
            1D array of values (probability stored in each pixel)
        nside : int
           skymap resolution
        """      
        
        self.hpx = hp.read_map(infile, verbose = False)
        npix = len(self.hpx)
        self.nside = hp.npix2nside(npix)
        
        return self.hpx, self.nside
 
    def ipixs_in_percentage(self, percentage):
        """Finding ipix indices confined in a given percentage.
        
        Input parameters
        ----------------
        percentage : float
                 fractional percentage from 0 to 1  
        
        Return
        ------- 
        ipixs : list
              indices of pixels
        """
        
        sort = sorted(self.hpx, reverse = True)
        cumsum = np.cumsum(sort)
        index, value = min(enumerate(cumsum), key = lambda x: abs( x[1] - percentage ))

        index_hpx = range(0, len( self.hpx ))
        hpx_index = np.c_[self.hpx, index_hpx]

        sort_2array = sorted(hpx_index, key = lambda x: x[0], reverse = True)
        value_contour = sort_2array[0:index]

        j = 1 
        table_ipix_contour = [ ]

        for i in range (0, len(value_contour)):
            ipix_contour = int(value_contour[i][j])
            table_ipix_contour.append(ipix_contour)
    
        self.ipixs = table_ipix_contour
          
        return self.ipixs
     
    def sky_coords(self):
        """Creating an astropy.table with RA[deg] and DEC[deg] ipix positions
        
        Return
        ------- 
        contour_ipix : 
                    sky coords in degrees
        """
       
        # from index to polar coordinates
        theta, phi = hp.pix2ang(self.nside, self.ipixs)

        # converting these to right ascension and declination in degrees
        ra = np.rad2deg(phi)
        dec = np.rad2deg(0.5 * np.pi - theta)
        
        # creating an astropy.table with RA[deg] and DEC[deg]
        from astropy.table import Table
        self.contour_ipix = Table([ra, dec], names = ('RA[deg]', 'DEC[deg]'), 
                             meta = {'ipix': 'ipix table'})
     
        return self.contour_ipix
    
    def moc_order(self):
        """Setting MOC order.
        
        Return
        ------- 
        moc_order : int
              
        """       
        
        self.moc_order = int(log( self.nside, 2))
     
        return self.moc_order

    def create_moc(self):
        """Creating a MOC map from the contour_ipix table."""
        
        self.moc = MOC.from_table(self.contour_ipix, 'RA[deg]', 'DEC[deg]',
                                  self.moc_order)

        return self.moc

    def write_moc(self, percentage, short_name):
        """Writing MOC file in fits format.
        
        Input parameters
        ----------------
        percentage : float
                 fractional percentage from 0 to 1 converted into a string
        short_name : str
                 file output
        """
        
        return self.moc.write(short_name + '_MOC_' + str(percentage), format = 'fits')
    
    def contour_plot(self, infile, percentage, short_name=''):
        """Creating/Writing a MOC contour region at a fixed level of probability.
        
        Input parameters
        ---------------
        infile : string
              LVC probability sky localization in healpix format
        percentage : float
                 fractional percentage from 0 to 1
        """
        
        self.read_skymap(infile)
        self.ipixs_in_percentage(percentage)
        self.sky_coords()
        self.moc_order()
        self.create_moc()
